# Utilizando o Apache Spark e demais ferramentas correlatas, além do grafo construído na [lista 5](https://github.com/jsantos93/grafos/tree/master/atividade5), implemente as seguintes operações:
1. Determine o nó central através do grau.
2. Determine o nó central por centralidade utilizando a distância de Wasserman e a distância harmônica (consultar material).
3. Determine o nó central por intermediação.

In [1]:
from graphframes import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf

In [2]:
conf = SparkConf().setAppName('myapp').setMaster('local')
sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)
nodes = spark.read.csv('data/transport-nodes.csv', header=True)
rels = spark.read.csv('data/transport-relationships.csv', header=True)
g = GraphFrame(nodes, rels)

## 1. Determine o nó central através do grau.

In [7]:
total_degree = g.degrees
in_degree = g.inDegrees
out_degree = g.outDegrees

In [10]:
(total_degree.join(in_degree, "id", how="left")
 .join(out_degree, "id", how="left")
 .fillna(0)
 .sort("inDegree", ascending=False)
 .show())

+----------------+------+--------+---------+
|              id|degree|inDegree|outDegree|
+----------------+------+--------+---------+
|        Igarassu|     5|       3|        2|
|       Salgueiro|     3|       3|        0|
|             Exu|     5|       2|        3|
|         Caruaru|     2|       2|        0|
|        Ouricuri|     3|       2|        1|
|       Instancia|     3|       2|        1|
|      Arco Verde|     3|       2|        1|
|         Amaraji|     3|       2|        1|
|         Surubim|     2|       1|        1|
|         Carpina|     2|       1|        1|
|       Petrolina|     2|       1|        1|
|      Chã Grande|     2|       1|        1|
|     Rio Formoso|     3|       1|        2|
|   Serra Talhada|     4|       1|        3|
|           Orobó|     2|       1|        1|
|Lagoa de Itaenga|     2|       1|        1|
|       Araripina|     3|       0|        3|
|      Feira Nova|     1|       0|        1|
|          Cupira|     1|       0|        1|
|      Par

## 2. Determine o nó central por centralidade utilizando a distância de Wasserman e a distância harmônica (consultar material).

In [5]:
import networkx as nx
import pandas as pd
import json

df = pd.read_csv("data/transport-relationships.csv")

In [6]:
Graphtype = nx.DiGraph()
gnx = nx.from_pandas_edgelist(df, edge_attr='cost', create_using=Graphtype)

In [8]:
result = nx.closeness_centrality(gnx)
print(json.dumps(result, indent=2))
print(max(result))

{
  "Araripina": 0.0,
  "Caruaru": 0.19160260201064458,
  "Igarassu": 0.23766447368421054,
  "Salgueiro": 0.2623481781376518,
  "Instancia": 0.10526315789473684,
  "Carpina": 0.09473684210526315,
  "Petrolina": 0.0935672514619883,
  "Serra Talhada": 0.2083633741888969,
  "Arco Verde": 0.2578055307760928,
  "Ouricuri": 0.24935289042277825,
  "Surubim": 0.09398496240601503,
  "Exu": 0.2083633741888969,
  "Cupira": 0.0,
  "Orob\u00f3": 0.05263157894736842,
  "Rio Formoso": 0.07017543859649122,
  "Amaraji": 0.19253830779480346,
  "Feira Nova": 0.0,
  "Parnamirim": 0.0,
  "Lagoa de Itaenga": 0.17483363581367212,
  "Ch\u00e3 Grande": 0.15059927045336113
}
Surubim


In [9]:
result = nx.harmonic_centrality(gnx)
print(json.dumps(result, indent=2))
print(max(result))

{
  "Araripina": 0,
  "Caruaru": 5.688492063492065,
  "Igarassu": 6.78452380952381,
  "Salgueiro": 7.33452380952381,
  "Instancia": 2.0,
  "Carpina": 2.0,
  "Petrolina": 2.1666666666666665,
  "Serra Talhada": 5.135714285714288,
  "Arco Verde": 6.5166666666666675,
  "Ouricuri": 6.409523809523811,
  "Surubim": 2.333333333333333,
  "Exu": 5.978968253968255,
  "Cupira": 0,
  "Orob\u00f3": 1.0,
  "Rio Formoso": 1.5,
  "Amaraji": 5.712301587301588,
  "Feira Nova": 0,
  "Parnamirim": 0,
  "Lagoa de Itaenga": 4.445238095238095,
  "Ch\u00e3 Grande": 4.009126984126984
}
Surubim


## 3. Determine o nó central por intermediação.

In [11]:
result = nx.betweenness_centrality(gnx)
print(json.dumps(result, indent=2))
print(max(result))

{
  "Araripina": 0.0,
  "Caruaru": 0.0,
  "Igarassu": 0.1388888888888889,
  "Salgueiro": 0.0,
  "Instancia": 0.04678362573099415,
  "Carpina": 0.07602339181286549,
  "Petrolina": 0.09941520467836257,
  "Serra Talhada": 0.2573099415204678,
  "Arco Verde": 0.2807017543859649,
  "Ouricuri": 0.25146198830409355,
  "Surubim": 0.11695906432748537,
  "Exu": 0.17982456140350878,
  "Cupira": 0.0,
  "Orob\u00f3": 0.03216374269005848,
  "Rio Formoso": 0.05847953216374269,
  "Amaraji": 0.1286549707602339,
  "Feira Nova": 0.0,
  "Parnamirim": 0.0,
  "Lagoa de Itaenga": 0.049707602339181284,
  "Ch\u00e3 Grande": 0.02631578947368421
}
Surubim
